In [13]:
import torch
import pandas as pd
from PIL import Image

#data format setup:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
input_data = torch.tensor([0,0,0,0,0,0,0,0,0], dtype=torch.float)

def play_ground(h, w, agent_a, agent_b, agent_c, agent_d, ball):
    rows = h
    columns = w * 3 + 2
    init_ground = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.float)
    for i in range(0, rows):
        for j in [w, -w - 1]:
            init_ground[i][j] = 1
    for i in [agent_a[0]]:
        for j in range(agent_a[1] + w, agent_a[3] + 1 + w):
            init_ground[i][j] = 1
    for i in [agent_b[0]]:
        for j in range(agent_b[1] + w, agent_b[3] + 1 + w):
            init_ground[i][j] = 1
    for i in [agent_c[0]]:
        for j in range(agent_c[3] + w, agent_c[1] + 1 + w):
            init_ground[i][j] = 1
    for i in [agent_d[0]]:
        for j in range(agent_d[3] + w, agent_d[1] + 1 + w):
            init_ground[i][j] = 1
    init_ground[ball[0]][ball[1]] = 2
    return init_ground

def agent_ball(perception, state):
    rows = 8
    columns = 3
    reference_matrix = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.int)
    reference_matrix[0][1] = -1
    reference_matrix[0][2] = -1
    reference_matrix[1][1] = -1
    reference_matrix[1][2] = 0
    reference_matrix[2][1] = -1
    reference_matrix[2][2] = 1
    reference_matrix[3][1] = 0
    reference_matrix[3][2] = 1
    reference_matrix[4][1] = 1
    reference_matrix[4][2] = 1
    reference_matrix[5][1] = 1
    reference_matrix[5][2] = 0
    reference_matrix[6][1] = 1
    reference_matrix[6][2] = -1
    reference_matrix[7][1] = 0
    reference_matrix[7][2] = -1

    if (state[2] * state[3]) == 0:
        if perception == [1, 1, 1]:
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        elif perception == [0, 1, 1]:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
                    print(perception)
                    print(start_point)
            new_speed_x = reference_matrix[start_point - 2][1]
            new_speed_y = reference_matrix[start_point - 2][2]
        elif perception == [1, 1, 0]:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
            new_speed_x = reference_matrix[(start_point + 2)% 8][1]
            new_speed_y = reference_matrix[(start_point + 2)% 8][2]
        elif perception == [0, 1, 0]:
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        else:
            new_speed_x = state[2]
            new_speed_y = state[3]
    else:
        if (perception[0] == 1) & (perception[2] == 1):
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        elif perception == [0, 1, 0]:
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        elif perception[2] == 1:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
                    print(perception)
                    print(start_point)
            new_speed_x = reference_matrix[start_point - 2][1]
            new_speed_y = reference_matrix[start_point - 2][2]
        elif perception[0] == 1:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
            new_speed_x = reference_matrix[(start_point + 2)% 8][1]
            new_speed_y = reference_matrix[(start_point + 2)% 8][2]
        else:
            new_speed_x = state[2]
            new_speed_y = state[3]

    action = [new_speed_x, new_speed_y]
    state = [state[0] + new_speed_x, state[1] + new_speed_y, new_speed_x, new_speed_y]
    return (action, state)


def func_perspection(play_ground, state):
    rows = 8
    columns = 3
    reference_matrix = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.int)
    reference_matrix[0][1] = -1
    reference_matrix[0][2] = -1
    reference_matrix[1][1] = -1
    reference_matrix[1][2] = 0
    reference_matrix[2][1] = -1
    reference_matrix[2][2] = 1
    reference_matrix[3][1] = 0
    reference_matrix[3][2] = 1
    reference_matrix[4][1] = 1
    reference_matrix[4][2] = 1
    reference_matrix[5][1] = 1
    reference_matrix[5][2] = 0
    reference_matrix[6][1] = 1
    reference_matrix[6][2] = -1
    reference_matrix[7][1] = 0
    reference_matrix[7][2] = -1
    for i in range(0,8):
        reference_matrix[i][0] = play_ground[state[0] + reference_matrix[i][1]][state[1] + reference_matrix[i][2]]
        if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
            start_point = i
    #print(reference_matrix)
    if (state[2] * state[3]) == 0:
        new_perception = [reference_matrix[start_point - 2][0], reference_matrix[start_point][0], reference_matrix[(start_point + 2)% 8][0]]
    else:
        new_perception = [reference_matrix[start_point - 1][0], reference_matrix[start_point][0], reference_matrix[(start_point + 1)% 8][0]]
    #print(new_perception)
    return new_perception

def agent_a(perception, state):
    action = 0
    state = 0
    return (action, state)



if __name__ == "__main__":
    #print(play_ground(12,10,[-1, 1, -1, 3],[-1, 4, -1, 6], [0, 6, 0, 4], [0, 3, 0, 1], [3, 3]))
    #New_play_ground = play_ground(12,10,[-1, 1, -1, 3],[-1, 4, -1, 6], [0, 6, 0, 4], [0, 3, 0, 1], [3, 3])
    #print(agent_ball([1, 0, 0], [5, 15, 1, 1]))
    current_state = [5, 16, 0, -1]
    #New_action, New_state = agent_ball([0, 0, 0], [5, 16, 1, 1])
    for i in range(0,15):
        Current_play_ground = play_ground(12,10,[-1, 1, -1, 3],[-1, 4, -1, 6], [0, 6, 0, 4], [0, 3, 0, 1], current_state[0: 2])
        #print(Current_play_ground)
        Current_perception = func_perspection(Current_play_ground, current_state)
        New_action, New_state = agent_ball(Current_perception, current_state)
        current_state = New_state
        #print(New_action)
        #print(New_state)
        #print(new_perception)
        """
        #print(New_play_ground)
        export_output = Current_play_ground.clone()
        export_output = export_output.cpu().detach().numpy()
        export_output = export_output * 120
        img = Image.fromarray(export_output)
        #img.save('my.png')
        img.show()
    #print(func_perspection(New_play_ground, New_state))
        """
        


In [16]:
"""
crop Matrix for 4 Agents
action: 1,2,3, array
RL model


draft: 3 output: score: 1, 0, -1???
draft: DL model

completed: save and load 3D data into local file (mat file)

"""

import numpy as np
import scipy.io

# Some test data
x = np.arange(200).reshape((4,5,10))

# Specify the filename of the .mat file
matfile = 'test_mat.mat'

# Write the array to the mat file. For this to work, the array must be the value
# corresponding to a key name of your choice in a dictionary
scipy.io.savemat(matfile, mdict={'out': x}, oned_as='row')

# For the above line, I specified the kwarg oned_as since python (2.7 with 
# numpy 1.6.1) throws a FutureWarning.  Here, this isn't really necessary 
# since oned_as is a kwarg for dealing with 1-D arrays.

# Now load in the data from the .mat that was just saved
matdata = scipy.io.loadmat(matfile)

# And just to check if the data is the same:
assert np.all(x == matdata['out'])

print(matdata['out'])

[[[  0   1   2   3   4   5   6   7   8   9]
  [ 10  11  12  13  14  15  16  17  18  19]
  [ 20  21  22  23  24  25  26  27  28  29]
  [ 30  31  32  33  34  35  36  37  38  39]
  [ 40  41  42  43  44  45  46  47  48  49]]

 [[ 50  51  52  53  54  55  56  57  58  59]
  [ 60  61  62  63  64  65  66  67  68  69]
  [ 70  71  72  73  74  75  76  77  78  79]
  [ 80  81  82  83  84  85  86  87  88  89]
  [ 90  91  92  93  94  95  96  97  98  99]]

 [[100 101 102 103 104 105 106 107 108 109]
  [110 111 112 113 114 115 116 117 118 119]
  [120 121 122 123 124 125 126 127 128 129]
  [130 131 132 133 134 135 136 137 138 139]
  [140 141 142 143 144 145 146 147 148 149]]

 [[150 151 152 153 154 155 156 157 158 159]
  [160 161 162 163 164 165 166 167 168 169]
  [170 171 172 173 174 175 176 177 178 179]
  [180 181 182 183 184 185 186 187 188 189]
  [190 191 192 193 194 195 196 197 198 199]]]
